#### Modifying Linear Search (WASM)

In [ ]:
def runwasm(wasmfile):
    from IPython.display import display, Javascript
    display(Javascript("""
    const params = {
        P0lib: {
            write: i => element.append(i + ' '),
            writeln: () => element.append(document.createElement('br')),
            read: () => window.prompt()
        }
    }
    
    var wasmByteString = \"""" + str(open(wasmfile, "rb").read()) + """\"; // pass the wasm file to JavaScript as byte string
    wasmByteString = wasmByteString.substring(2, wasmByteString.length - 1); // remove the byte literals b'...'
    const wasmArrayBuffer = new Uint8Array(wasmByteString.length); // convert the binary string to ArrayBuffer
    for (let i = 0; i < wasmByteString.length; i++)
      wasmArrayBuffer[i] = wasmByteString.charCodeAt(i);
    
    WebAssembly.compile(wasmArrayBuffer.buffer) // compile (sharable) code
        .then(module => WebAssembly.instantiate(module, params)) // create an instance with memory
        // .then(instance => instance.exports.program()); // run the main program; not needed if a start function is specified
     """))

In [ ]:
def runpywasm(wasmfile):
    import pywasm
    def write(s, i): print(i, end=' ')
    def writeln(s): print()
    def read(s): return int(input())
    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read}})

In [ ]:
from wasmer import engine, Store, Module, Instance, ImportObject, Function
from wasmer_compiler_cranelift import Compiler

def runwasmer(wasmfile):
    def write(i: int): print(i, end=' ')
    def writeln(): print()
    def read() -> int: return int(input()) 
    store = Store(engine.JIT(Compiler))
    module = Module(store, open(wasmfile, 'rb').read())
    import_object = ImportObject()
    import_object.register("P0lib", {"write": Function(store, write),
                                     "writeln": Function(store, writeln),"read": Function(store, read)})
    instance = Instance(module, import_object)

---

Compile the P0 program below by running the next two cells:

In [ ]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString

In [ ]:
compileString("""
const N = 5
var a: [1 .. N] → integer
program linearsearch
    var i, x: integer
        x ← read(); i := 0 // read number to be searched
        while i < N do i := i + 1; a[i] ← read()  // read array elements
        while (i > 0) and (a[i] ≠ x) do i := i - 1
        writeln(); write(i) // write index of last occurrence of x or 0 if not found
""", 'linearsearch.wat')

 You can convert `linearsearch.wat` to a binary file and run it using one of the three methods above for running WebAssembly in Jupyter:

In [ ]:
!wat2wasm linearsearch.wat

In [ ]:
runwasm("linearsearch.wasm")

In [ ]:
runpywasm("linearsearch.wasm")

In [ ]:
runwasmer("linearsearch.wasm")

Now inspect the generated code by running the cell below or leaving out the last parameter of `compileString` and using `print` to print the resulting string. Explain the role of each line!

In [ ]:
!cat linearsearch.wat

Your answer here.

*Instructor's Answer:*
```
(module
(import "P0lib" "write" (func $write (param i32))) ;; import standard library functions
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(global $_memsize (mut i32) i32.const 20) ;; initialize memory pointer
(func $program
(local $i i32) ;; declare local variables
(local $x i32)
(local $0 i32)
call $read 
local.set $x ;; x ← read();
i32.const 0 
local.set $i ;; i := 0
loop ;; declare loop
local.get $i
i32.const 5
i32.lt_s ;; i < N
if
local.get $i
i32.const 1
i32.add
local.set $i ;; i := i + 1
local.get $i
i32.const 1
i32.sub ;; subtract 1 from i (because array starts at 1)
i32.const 4
i32.mul ;; multiply i by 4
i32.const 0 ;; offset of 0
i32.add ;; calculate address of a[i]
call $read
i32.store ;;  a[i] ← read()
br 1
end
end
loop
local.get $i
i32.const 0
i32.gt_s ;; i > 0
if (result i32) ;; declare if statement with i32 return type
local.get $i
i32.const 1
i32.sub ;; i - 1
i32.const 4
i32.mul
i32.const 0
i32.add
i32.load ;; a[i]
local.get $x
i32.ne ;; a[i] ≠ x
else
i32.const 0
end
if
local.get $i
i32.const 1
i32.sub
local.set $i ;; i := i - 1
br 1
end
end
call $writeln ;; writeln()
local.get $i
call $write ;; write(i)
)
(memory 1) ;; set up memory size 
(start $program) ;; call main program
)
```

Perform the following modifications of the above program; it is repeated below:

1. Make variables `i` and `x` global rather than local to the main program. What changes do you observe in the generated code?
2. Make array `a` a local variable of the main program. What changes do you observe in the generated code?
3. Make constant `N` a local constant of the main program. What changes do you observe in the generated code?

In [ ]:
wasm = compileString("""
const N = 5
var a: [1 .. N] → integer
program linearsearch
    var i, x: integer
        x ← read(); i := 0 // read number to be searched
        while i < N do i := i + 1; a[i] ← read()  // read array elements
        while (i > 0) and (a[i] ≠ x) do i := i - 1
        writeln(); write(i) // write index of last occurrence of x or 0 if not found
""")
print(wasm)

Your answer here.

*Instructor's Answer.*

1. When `i` and `x` are local to the main program, they are declared in WebAssembly as local variables to `func $program`, i.e. space for the variables is allocated on the stack when `func $program` is called. When `i` and `x` are global variables, they are declared as `global` in WebAssembly, i.e. space for them is allocated when the a WebAssembly instance of the module is created.
2. In both variants, array `a` is allocated in WebAssembly memory, i.e. on the heap. When `a` is a global variable, its address is determined statically by the compiler and `memsize` is initialized to the size of all statically allocated variables; as `a` is the only global variable, `memsize` is initially the size of `a`. When `a` is a local variable, the space for `a` is allocated dynamically when the main program starts by incrementing `memsize`, which is initialized to `0` instead.
3. There is no change, in P0 constant names are replaced by their values when the code is generated.